In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("train.csv", sep=',', encoding='latin-1')

In [ ]:
df.sample(10)

,qid,question_text,target
180,00083b24297e4e627776,If an average NFL team obtained a kicker who c...,0.0
8413,01a356e27d7902fff18c,What are the main features of the world's grea...,0.0
49226,09a2aed3b8786f46848b,Why are French people do not answer tourists e...,1.0
37918,076940ee1daa90daf318,What do you think of Bitcoin Gold (BTG)? Will ...,0.0
54759,0abf21602d8fe3f46d41,What is the difference between assertive sente...,0.0
9526,01dc87c266d59a317cf6,What are the achievements of Arvind Kejriwal a...,0.0
16625,03435b6bf17501177299,How do I interpret salary grades that only lis...,0.0
20447,0400a83f10903a191600,What is the Champions League?,0.0
38680,0790d10c5e46461aaec9,My dad went to prison for something another pe...,0.0
25313,04f78f36eb2027600c40,What previous knowledge do I need to have for ...,0.0


In [ ]:
for index, row in enumerate(df.isnull()["target"]):
  if row == True:
    print(index)

55264


In [ ]:
df.drop([55264],axis=0,inplace=True)

In [ ]:
df.isnull().sum()

qid              0
question_text    0
target           0
dtype: int64

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model

from nltk import word_tokenize

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [ ]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2022-05-03 06:49:43--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2022-05-03 06:49:43--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2022-05-03 06:49:43--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [ ]:
!unzip /content/glove.42B.300d.zip 

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [ ]:
!rm /content/glove.42B.300d.zip

In [ ]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [ ]:
len(embeding_index['provide'])

300

In [ ]:
len(embeding_index)

1917494

In [ ]:
X=df['question_text']
Y=df['target']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((44211,), (11053,), (44211,), (11053,))

In [ ]:
class_weight=class_weight.compute_class_weight('balanced' ,classes = np.unique(y_train), y = y_train)
print(class_weight)

[0.53245737 8.20241187]


In [ ]:
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.5324573658348588, 1: 8.202411873840445}

In [ ]:
x_train

39938    Do other countries have seasonal "Halloween" s...
21185    Why native Hindi speakers wants south Indians ...
17019    I think that I know an idea to help cure cance...
484      Is there a way to report a business review of ...
12562    Where can I find wholesale suppliers of Korean...
                               ...                        
14486    Does Tybalt ever confront or meet Romeo direct...
47050    What are the examples for law, ethics and posi...
44522    How is a Radeon Pro 555 that has 2GB of GDDR5 ...
21115    Which is the best college for interior design ...
38238    How do I become a data scientist with no softw...
Name: question_text, Length: 44211, dtype: object

In [ ]:
from nltk import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

97


In [ ]:
np.percentile(sent_lens,99)

44.0

In [ ]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [ ]:
tk.word_index

In [ ]:
vocab_size

35612

In [ ]:
seq_train_matrix.shape

(44211, 44)

In [ ]:
seq_train

In [ ]:
x_train.shape

(44211,)

In [ ]:
len(seq_train),len(seq_test)

(44211, 11053)

In [ ]:
seq_train_matrix.shape, seq_test_matrix.shape

((44211, 44), (11053, 44))

In [ ]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [ ]:
 len(embedding_matrix)

35613

In [ ]:
embedding_matrix.shape

(35613, 300)

In [ ]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.38066   , -0.92199999, -0.16158   , ...,  0.079318  ,
        -0.014611  , -0.2762    ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 44)]              0         
                                                                 
 embedding (Embedding)       (None, 44, 300)           10683900  
                                                                 
 lstm (LSTM)                 (None, 512)               1665024   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896 

In [ ]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(seq_train_matrix,y_train,
              epochs=15,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=1000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint],
              verbose=2)

Epoch 1/15

Epoch 1: val_loss improved from inf to 0.34232, saving model to /content/weights-01-0.3423.h5
45/45 - 390s - loss: 0.4208 - accuracy: 0.7958 - val_loss: 0.3423 - val_accuracy: 0.8488 - 390s/epoch - 9s/step
Epoch 2/15

Epoch 2: val_loss improved from 0.34232 to 0.17963, saving model to /content/weights-02-0.1796.h5
45/45 - 385s - loss: 0.3284 - accuracy: 0.8524 - val_loss: 0.1796 - val_accuracy: 0.9277 - 385s/epoch - 9s/step
Epoch 3/15

Epoch 3: val_loss did not improve from 0.17963
45/45 - 380s - loss: 0.3120 - accuracy: 0.8687 - val_loss: 0.3450 - val_accuracy: 0.8470 - 380s/epoch - 8s/step
Epoch 4/15

Epoch 4: val_loss did not improve from 0.17963
45/45 - 378s - loss: 0.2804 - accuracy: 0.8712 - val_loss: 0.2854 - val_accuracy: 0.8675 - 378s/epoch - 8s/step
Epoch 5/15

Epoch 5: val_loss did not improve from 0.17963
45/45 - 387s - loss: 0.2613 - accuracy: 0.8762 - val_loss: 0.2202 - val_accuracy: 0.9040 - 387s/epoch - 9s/step
Epoch 5: early stopping


In [ ]:
best_model = load_model('/content/weights-02-0.1796.h5')

In [ ]:
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 44)]              0         
                                                                 
 embedding (Embedding)       (None, 44, 300)           10683900  
                                                                 
 lstm (LSTM)                 (None, 512)               1665024   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896 

In [ ]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9284649661446333

In [ ]:
best_model.save('./spam_filter_model_quora_questions.h5')